In [1]:
## uncomment for the first run
# import Pkg
# Pkg.add("PyCall")
# Pkg.add("Conda")
# ENV["PYTHON"] = "/opt/anaconda3/envs/turi/bin/python"
# /opt/anaconda3/envs/turi/lib/python3.6/site-packages/
# Pkg.build("PyCall")

In [2]:
using PyCall
tc = pyimport("turicreate")

PyObject <module 'turicreate' from '/opt/anaconda3/envs/turi/lib/python3.6/site-packages/turicreate/__init__.py'>

In [3]:
data_path="./DATA/processed/uber.csv"
data = tc.SFrame(data_path)
println("data is a ", typeof(data))

Finished parsing file /Users/uki/REPOS/MarketIndicators.jl/DATA/processed/uber.csv
Parsing completed. Parsed 100 lines in 0.056482 secs.
------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
Finished parsing file /Users/uki/REPOS/MarketIndicators.jl/DATA/processed/uber.csv
Parsing completed. Parsed 1744 lines in 0.010981 secs.
data is a PyObject


In [4]:
row_count = size(data)[1]

1744

In [5]:
# Do not take initial year data as averages are not complete
get(data, 1743)

Dict{Any,Any} with 24 entries:
  "MFC_MPI_Avg180"        => 27.84
  "DJIA_Avg030"           => 111.1
  "HOUSE_SRT_MM_Avg090"   => 59.16
  "Date"                  => "2020-10-20"
  "DJIA_Value"            => 115.0
  "MFC_MPI_Value"         => 74.0
  "DJIA_Avg180"           => 98.59
  "DJIA_Avg005"           => 116.4
  "HOUSE_SRT_MM_Avg180"   => 4.14
  "MFC_MPI_Avg365"        => 22.17
  "HOUSE_SRT_MM_Avg365"   => 9.5
  "ISM_MFC_EMP_Value"     => 44.0
  "ISM_MFC_EMP_Avg365"    => -6.67
  "HOUSE_SRT_MM_Value"    => 19.0
  "DJIA_Avg090"           => 109.08
  "Day"                   => 737718
  "MFC_MPI_Avg090"        => 69.84
  "DJIA_Avg365"           => 99.24
  "HOUSE_SRT_MM_Original" => 1.9
  "DJIA_Original"         => 28575.0
  "ISM_MFC_EMP_Avg090"    => 15.37
  "ISM_MFC_EMP_Original"  => 49.6
  "ISM_MFC_EMP_Avg180"    => -31.44
  "MFC_MPI_Original"      => 55.4

In [6]:
# Make a train-test split
train_data, test_data = data.random_split(0.8)
println()

In [7]:
size(train_data)
size(test_data)

(357,)

In [8]:
column_to_predict = "DJIA_Original"

model = tc.regression.create( 
    train_data, 
    target = column_to_predict, 
    features = [
                 "DJIA_Avg005"
                ,"DJIA_Avg030"
                ,"DJIA_Avg090"
                ,"DJIA_Avg180"        
                ,"DJIA_Avg365"

                ,"ISM_MFC_EMP_Avg090"
                ,"ISM_MFC_EMP_Avg180"
                ,"ISM_MFC_EMP_Avg365"

                ,"HOUSE_SRT_MM_Value"
                ,"HOUSE_SRT_MM_Avg090"
                ,"HOUSE_SRT_MM_Avg180"
                ,"HOUSE_SRT_MM_Avg365"

                ,"MFC_MPI_Value"
                ,"MFC_MPI_Avg090"
                ,"MFC_MPI_Avg180"
                ,"MFC_MPI_Avg365"
            ], 
    validation_set="auto", 
    verbose=true
)

Automatically generating validation set from 5% of the data.
Linear regression:
--------------------------------------------------------
Number of examples          : 1327
Number of features          : 16
Number of unpacked features : 16
Number of coefficients    : 17
Starting Newton Method
--------------------------------------------------------
+-----------+----------+--------------+--------------------+----------------------+---------------------------------+-----------------------------------+
| Iteration | Passes   | Elapsed Time | Training Max Error | Validation Max Error | Training Root-Mean-Square Error | Validation Root-Mean-Square Error |
+-----------+----------+--------------+--------------------+----------------------+---------------------------------+-----------------------------------+
| 1         | 2        | 1.010805     | 1743.450375        | 1070.171629          | 216.223451                      | 281.031611                        |
+-----------+----------+-----------

PyObject Class                          : LinearRegression

Schema
------
Number of coefficients         : 17
Number of examples             : 1327
Number of feature columns      : 16
Number of unpacked features    : 16

Hyperparameters
---------------
L1 penalty                     : 0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 1.017

Settings
--------
Residual sum of squares        : 62040674.8527
Training RMSE                  : 216.2235

Highest Positive Coefficients
-----------------------------
(intercept)                    : 15723.082
DJIA_Avg005                    : 123.5016
DJIA_Avg180                    : 5.0173
MFC_MPI_Avg365                 : 4.2899
DJIA_Avg090                    : 2.3055

Lowest Negative Coefficients
----------------------------
DJIA_Avg030               

In [9]:
## Save predictions to an SArray
predictions = model.predict(test_data)
#predictions

PyObject dtype: float
Rows: 357
[16124.0690820535, 16051.182985577749, 16106.425161105835, 16223.924237392977, 16503.782138125003, 16191.965447024086, 16049.340885436586, 16001.164654040658, 16171.701436120027, 16390.035734366294, 16534.745748321173, 16837.980827076273, 16910.56581923466, 17190.477220546098, 17211.821474909622, 17182.711424149133, 17277.76130893489, 17329.143630776223, 17489.878080423558, 17583.475315818818, 17615.77156750971, 17731.263629054603, 17704.011750995793, 17744.633762161666, 17908.646512880612, 17956.00405345644, 17979.71275589384, 18043.53582555946, 17814.393708649273, 17814.308850489622, 17789.593231931005, 17770.124470258314, 17771.37518427503, 17614.232654178537, 17763.773605438633, 17839.269756386602, 17938.004486531358, 17863.16940745228, 17885.111083340726, 17904.921098960498, 17908.09389101022, 17640.850836903075, 18220.898983245337, 18531.50334495444, 18628.893494993645, 18464.616939588323, 18438.260517108418, 18406.942807656033, 18571.33130576751, 

In [10]:
initio = 0                    # latin: start
gradus = 30                   # latin: step
finem  = size(predictions)[1] # latin: end

println(column_to_predict, " ", finem)

for id in initio:gradus:finem
    
    a = get(predictions, id)
    a = round(a, digits=2)
    
    b = get(test_data, column_to_predict )
    b = get(b, id )
    b = round(b, digits=2)

    println( "predicted ", a, "\t, but actual value was \t", b , "\t difference is ",  round(b-a, digits=2)  ) # dict
end

DJIA_Original 357
predicted 16124.07	, but actual value was 	16038.59	 difference is -85.48
predicted 17789.59	, but actual value was 	17744.54	 difference is -45.05
predicted 18191.5	, but actual value was 	18168.76	 difference is -22.74
predicted 20673.87	, but actual value was 	20612.27	 difference is -61.6
predicted 22050.11	, but actual value was 	22038.97	 difference is -11.14
predicted 26497.29	, but actual value was 	26616.71	 difference is 119.42
predicted 25388.4	, but actual value was 	25364.37	 difference is -24.03
predicted 25606.38	, but actual value was 	25608.71	 difference is 2.33
predicted 26593.4	, but actual value was 	26695.96	 difference is 102.56
predicted 26359.93	, but actual value was 	26514.62	 difference is 154.69
predicted 28629.33	, but actual value was 	28701.66	 difference is 72.33
predicted 24119.09	, but actual value was 	24599.5	 difference is 480.41


In [11]:
#TODO: write this in a loop to select the best model
# Evaluate the model and save the results into a dictionary
results = model.evaluate( test_data ) #test_data[0:2531]
results

Dict{Any,Any} with 2 entries:
  "rmse"      => 195.261
  "max_error" => 1291.43

In [12]:
rmse = results["rmse"]
rmse = round(rmse, digits=2)

max_error = results["max_error"]
max_error = round(max_error, digits=2)

println( "max_error: ", max_error, ", rmse: ", rmse )

max_error: 1291.43, rmse: 195.26


In [13]:
# Export to Core ML
model.export_coreml("./DATA/models/^DJI.mlmodel")